## 3.3 Synthetic Regression Data

In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [11]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=6):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

In [17]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2, batch_size=11)

In [13]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([ 0.7979, -1.6054]) 
label: tensor([11.2404])


In [14]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

In [18]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([11, 2]) 
y shape: torch.Size([11, 1])


In [19]:
print(X)

tensor([[ 0.4009, -0.9727],
        [-0.5365, -1.0499],
        [-0.5279,  0.8842],
        [ 0.6653,  0.9449],
        [ 0.5579, -0.1440],
        [-0.6001,  0.0905],
        [-0.0760,  0.6949],
        [-0.2599,  0.4157],
        [-0.8851,  0.5849],
        [ 0.8732,  1.6391],
        [-0.8605, -1.1193]])


In [10]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

In [30]:
import random
random.seed(1)

for i in range(5): 
    print(random.randint(0, 10))

2
9
1
4
1
